# loading libraries

In [37]:
import pandas as pd
import numpy as np
import os
import glob
import warnings
warnings.filterwarnings("ignore")

# Data Loading 

In [38]:
df =  pd.read_excel('../RawData/Quantity_forecast_new_daily.xlsx',sheet_name='DAILY')
df.head()

,sku,date,region,order_id,net_revenue,quantity
0,5213,2022-11-23,BE_B2B,0,5.90,2
1,1000001,2022-02-02,BE_SHOP_DACH,0,0.00,3
2,1000001,2022-02-03,BE_SHOP_DACH,0,0.00,2
3,1000009,2022-07-22,BE_SHOP_INT,0,0.00,1
4,1000011,2023-03-17,BE_B2B,0,-151.74,1


In [39]:
df.describe()

,date,order_id,net_revenue,quantity
count,63081,63081.000000,63081.000000,63081.000000
mean,2022-11-10 05:27:58.013981952,6.235126,130.450507,10.298156
min,2021-12-31 00:00:00,0.000000,-299.065421,0.000000
25%,2022-06-20 00:00:00,1.000000,23.601896,2.000000
50%,2022-11-20 00:00:00,2.000000,60.358879,3.000000
75%,2023-04-03 00:00:00,4.000000,116.914359,5.000000
max,2023-08-15 00:00:00,1024.000000,89812.800000,17820.000000
std,NaN,20.183364,645.565528,112.597124


# Filteration the data 
I will do the time series analysis only these SKU ID and Shop ID:

**Suggested sku'z**
* BE20319
* 8BE20090
* BE20223
* BE20207
* BE20044
* BE20209
* BE20206
* BE20198
* BE20196
* BE20222
* BE20313
* BE20320
* BE20293
* BE20263
* BE20335
* BE20182
* BE20191
* BE20288


**Shop ID**
* BE_SHOP_DACH
* BE_SHOP_INT

In [40]:
skus_id = ['BE20319','BE20090','BE20223','BE20207',
        'BE20044','BE20209','BE20206','BE20198',
        'BE20196','BE20222','BE20313','BE20320',
        'BE20293','BE20263','BE20335','BE20182',
        'BE20191','BE20288']
Shop_id = ['BE_SHOP_DACH', 'BE_SHOP_INT']
# filtering the dataframe base on suggested sku and shop id
filtered_df = df[(df['sku'].isin(skus_id) ) & (df['region'].isin(Shop_id)) & (df['net_revenue']!=0)  ]
# filtered_df = df[(df['sku'].isin(skus_id) ) & (df['region'].isin(Shop_id))]

print('shape of Filterted Dataframe : ',filtered_df.shape)

filtered_df.reset_index(inplace=True,drop=True)

display(filtered_df.head())
filtered_df.to_csv('../processedData/filteredDataSCM.csv',index=False)

shape of Filterted Dataframe :  (6811, 6)


,sku,date,region,order_id,net_revenue,quantity
0,BE20044,2022-04-15,BE_SHOP_DACH,1,31.696262,1
1,BE20044,2022-12-24,BE_SHOP_DACH,1,29.831776,1
2,BE20044,2022-12-10,BE_SHOP_DACH,1,37.289720,1
3,BE20044,2022-10-18,BE_SHOP_DACH,1,29.831776,1
4,BE20044,2022-05-28,BE_SHOP_DACH,1,37.289720,1


# EDA of Filtered Data

## Saving Dataframe for each sku with shop in csv files

In [41]:
grouped_df = filtered_df.groupby(['sku', 'region'])
meta_information = {
    'Data_id':[],
    'number_of_records_daily':[],
    'number_of_records_2023_daily':[],
    'number_of_records_2022_daily':[],
    'number_of_records_2021_daily':[],
    'Number_of_missing_records_daily':[],
    'starting_date':[],
    'Ending_date':[],
    'total_records_should_be':[]
}
for index, group_df in grouped_df:
    group_df = group_df.drop_duplicates()
    group_df['date'] = pd.to_datetime(group_df['date'],format='%Y-%m-%d')

    total_records = group_df.shape[0]
    group_df.reset_index(inplace=True,drop=True)
    
    group_df['year'] = group_df['date'].dt.year
    group_df['month'] = group_df['date'].dt.month
    group_df['day'] = group_df['date'].dt.day


    number_of_records_2023 = group_df[group_df['year'] == 2023].shape[0]
    number_of_records_2022 = group_df[group_df['year'] == 2022].shape[0]
    number_of_records_2021 = group_df[group_df['year'] == 2021].shape[0]

    # group_df = group_df[group_df['year'] != '2023']
    # group_df_descending = group_df.sort_values(by=['year', 'month','day'], ascending=[True, False,False])
    group_df_descending = group_df.sort_values(by='date', ascending=True)
    group_df_descending.reset_index(inplace=True,drop=True)

    group_df_descending.to_csv(f'../processedData/Sku_shop_based_data/{index[0]}_{index[1]}.csv',index=False)
    # Fillling missing value
    date_range = pd.date_range(start=group_df_descending['date'].min(), end=group_df_descending['date'].max(), freq='D')
    merged_df = pd.merge(group_df_descending, pd.DataFrame({'date': date_range}), how='right', on='date')

    number_of_missing_records = merged_df['quantity'].isna().sum()
    
    
    meta_information['Data_id'].append(f'{index[0]}_{index[1]}')
    meta_information['number_of_records_daily'].append(total_records)
    meta_information['number_of_records_2023_daily'].append(number_of_records_2023)
    meta_information['number_of_records_2022_daily'].append(number_of_records_2022)
    meta_information['number_of_records_2021_daily'].append(number_of_records_2021)
    meta_information['Number_of_missing_records_daily'].append(number_of_missing_records)
    meta_information['starting_date'].append(group_df_descending['date'].min())
    meta_information['Ending_date'].append(group_df_descending['date'].max())
    meta_information['total_records_should_be'].append(len(date_range.tolist()))


meta_information_df =  pd.DataFrame.from_dict(meta_information)
meta_information_df_sorted =meta_information_df.sort_values(by='number_of_records_daily',ascending=False)
meta_information_df_sorted.reset_index(inplace=True,drop=True)
meta_information_df_sorted.to_csv('../processedData/meta_information/meta_information_about_daily_records.csv',index=False)

In [42]:
meta_information_df_sorted

,Data_id,number_of_records_daily,number_of_records_2023_daily,number_of_records_2022_daily,number_of_records_2021_daily,Number_of_missing_records_daily,starting_date,Ending_date,total_records_should_be
0,BE20196_BE_SHOP_DACH,437,173,264,0,152,2022-01-03,2023-08-14,589
1,BE20222_BE_SHOP_INT,329,146,183,0,193,2022-03-10,2023-08-13,522
2,BE20090_BE_SHOP_DACH,323,170,153,0,242,2022-01-27,2023-08-14,565
3,BE20198_BE_SHOP_DACH,290,61,229,0,283,2022-01-03,2023-07-29,573
4,BE20191_BE_SHOP_INT,223,75,148,0,311,2022-01-03,2023-06-20,534
5,BE20198_BE_SHOP_INT,221,117,104,0,358,2022-01-06,2023-08-07,579
6,BE20263_BE_SHOP_DACH,196,158,38,0,108,2022-10-15,2023-08-14,304
7,BE20209_BE_SHOP_DACH,191,110,81,0,267,2022-05-13,2023-08-13,458
8,BE20182_BE_SHOP_DACH,188,20,168,0,235,2022-03-01,2023-04-27,423
9,BE20182_BE_SHOP_INT,186,77,109,0,128,2022-06-18,2023-04-27,314


# Visualization of each SKU with shop

In [66]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_scatter(df):
    os.makedirs('../plots',exist_ok=True)
    sku_id = df['sku'].unique().tolist()[0]
    region_id = df['region'].unique().tolist()[0]
    
    fig = make_subplots(rows=2,cols=1,shared_xaxes=True,vertical_spacing=0.09)
                        
    
    fig.add_trace(go.Scatter(x=df['date'], y=df['quantity'], mode='markers+lines',name='Quantity'),row=1,col=1)
    fig.add_trace(go.Scatter(x=df['date'], y=df['net_revenue'], mode='markers+lines',name='Revenue'),row=2,col=1)

    
    fig.update_layout(
        title='Scatter Plot of Quantity and Revenue vs Date',
        height=600,
        width=1800,
        font_size=14
    )
    fig.update_yaxes(title_text='Quantity', row=1, col=1)
    fig.update_yaxes(title_text='Revenue', row=2, col=1)
    
    fig.write_html(f'../plots/{sku_id}_{region_id}.html')


In [67]:
# calling for the visualization
sku_region_path_list = glob.glob('../processedData/Sku_shop_based_data/*.csv')
for path in sku_region_path_list:
    df = pd.read_csv(path)
    plot_scatter(df)

In [117]:
# making meta information for the desboard
sku_ids_list = glob.glob('../plots/*.html')
sku_ids = [sku.split('/')[2].split('_')[0] for sku in sku_ids_list]
region_ids = [ "_".join(sku.split('/')[-1].split('.')[0].split('_')[1:]) for sku in sku_ids_list]
region_ids = list(set(region_ids))



selected_skus = ['BE20196', 'BE20222', 'BE20090', 'BE20198', 'BE20191']
selected_sku_ids = {}
for sku_id in sku_ids:
    path_list = glob.glob('../plots/*.html')
    existing_sku = ["_".join(path.split('/')[-1].split('.')[0].split('_')[1:]) for path in path_list if sku_id in path]    
    if sku_id in selected_skus:
        selected_sku_ids[sku_id] = existing_sku
selected_sku_ids


{'BE20198': ['BE_SHOP_INT', 'BE_SHOP_DACH'],
 'BE20090': ['BE_SHOP_DACH', 'BE_SHOP_INT'],
 'BE20191': ['BE_SHOP_INT', 'BE_SHOP_DACH'],
 'BE20222': ['BE_SHOP_DACH', 'BE_SHOP_INT'],
 'BE20196': ['BE_SHOP_INT', 'BE_SHOP_DACH']}

In [122]:
selected_sku_ids['BE20198']

['BE_SHOP_INT', 'BE_SHOP_DACH']

In [121]:
list(selected_sku_ids.keys())

['BE20198', 'BE20090', 'BE20191', 'BE20222', 'BE20196']